In [ ]:
%pip install weaviate-client

# Connect to Weaviate Local Instance

In [105]:
import weaviate
import weaviate.classes.config as wc
import weaviate.classes.query as wq
from weaviate.classes.init import AdditionalConfig, Timeout
import os

client = weaviate.connect_to_local(
    # comment the line above and uncomment the one below if using docker
    #url="http://localhost:8080",
    headers={
        "X-OpenAi-Api-Key": "NOT_NEEDED_FOR_LM_STUDIO",
    },
    additional_config=AdditionalConfig(
        timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
    )
)

print("Weaviate Client is Ready: ", client.is_ready())
products = client.collections.get("product")
reviews = client.collections.get("review")

def print_product(o):
    print("Product Title: " + o.properties["title"])
    print("  Artist: " + o.properties["brand"])
    print("  ASIN: " + o.properties["asin"])
    print("  Categories: ")
    for c in o.properties["category"]:
        print("    " + c)
    print("  Price: " + o.properties["price"])
    print("  Description: ")
    for d in o.properties["description"]:
        print("    " + d)

def print_products(response_objects):
    seen_asin = []

    for o in response_objects:
        if o.properties["asin"] in seen_asin:
            continue
        seen_asin.append(o.properties["asin"])
        print_product(o)
        #print(f"\tDistance to query: {o.metadata.distance:.3f}")

def print_review(o):
    print(f"    Review Score: " + str(o.properties["overall"]))
    print(f"    Review Date: " + o.properties["unixReviewDate"].strftime("%Y-%m-%d"))
    print(f"    Reviewer Name: " + o.properties["reviewerName"])
    print(f"    Review Text: " + o.properties["reviewText"].replace("\n", "").replace("\r", ""))

def print_reviews(response_objects):
    count = 1
    seen_asin = []
    for o in response_objects:
        if o.properties["asin"] in seen_asin:
            continue
        seen_asin.append(o.properties["asin"])
        print(f"  Review {count}:")
        print_review(o)
        count += 1

collections = client.collections.list_all()
# Print all collection names
for name in collections:
    collection = client.collections.get(name)
    aggregation = collection.aggregate.over_all(total_count=True)
    print(f"{name}: {aggregation.total_count}")


Weaviate Client is Ready:  True
Product: 497683
Review: 1087778


/Users/ctindel/src/weaviate-demo/.venv/lib/python3.12/site-packages/weaviate/warnings.py:303: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


## Keyword Search

In [ ]:
def keyword_query(keyword):
    response = products.query.bm25(
        query=keyword, limit=5, return_metadata=wq.MetadataQuery(score=True)
    )

    print_products(response.objects)

keyword_query("charlie parker")

## Vector/Semantic Search

In [ ]:
def text_query(search_string):
    response = products.query.near_text(
        query=search_string,
        limit=5,
        return_metadata=wq.MetadataQuery(distance=True),
    )

    print_products(response.objects)

text_query("charlie parker")

## Semantic Search with Reviews

In [ ]:
def text_query_with_reviews(search_string):
    response = products.query.near_text(
        query=search_string,
        limit=5,
        return_metadata=wq.MetadataQuery(distance=True),
    )

    seen_asin = []
    for o in response.objects:
        if o.properties["asin"] in seen_asin:
            continue
        seen_asin.append(o.properties["asin"])
        print_product(o)
        review_response = reviews.query.bm25(
            query=o.properties["asin"],
            filters=wq.Filter.by_property("asin").equal(o.properties["asin"]),
            limit=1
        )
        print_reviews(review_response.objects)
        print("\n")

text_query_with_reviews("dire straits live")

## Semantic Search with Review Text Translated

In [111]:
LANGUAGE = "dutch"

def text_query_with_reviews(search_string):
    response = products.query.near_text(
        query=search_string,
        limit=2,
        return_metadata=wq.MetadataQuery(distance=True),
        return_properties=["title", "brand", "asin", "category", "price", "description"],
    )

    seen_product_asin = []
    for o in response.objects:
        if o.properties["asin"] in seen_product_asin:
            continue
        seen_product_asin.append(o.properties["asin"])
        print_product(o)
        review_response = reviews.generate.bm25(
            query=o.properties["asin"],
            filters=wq.Filter.by_property("asin").equal(o.properties["asin"]),
            limit=1,
            single_prompt="Translate this into " + LANGUAGE + ": {reviewText}"
        )
        review_count = 1
        seen_review_asin = []
        for o in review_response.objects:
            if o.properties["asin"] in seen_review_asin:
                continue
            seen_review_asin.append(o.properties["asin"])
            print(f"  Review {review_count}:")
            print_review(o)
            print("    Translated Review: " + o.generated.replace("\n", "").replace("\r", ""))
            review_count += 1
        print("\n")

text_query_with_reviews("dire straits live")

Product Title: Sultans of Swing Live in Germany Set
  Artist: Dire Straits
  ASIN: B001MYIPY6
  Categories: 
    CDs & Vinyl
    Rock
    Blues Rock
  Price: 
  Description: 
    2009 release containing a classic early live show from Mark Knopfler and Dire Straits, performed in Germany in 1979. This release captures the original band in their first brush of international success - Mark and Dave Knopfler on lead and rhythm guitars, John Illsley on bass, Pick Withers on drums. For all its newfound success, Dire Straits here puts on a carefully polished, musically serious show, full of the nuance and intelligence that characterized the band s memorable career in Rock. A very young Mark Knopfler is of course front and center, with his trademark Dylan-influenced vocals, seductive guitar playing, and dark and thoughtful songs. Those songs include beloved numbers from their breakout self-titled debut album ('Sultans Of Swing', Down To The Waterline, Six Blade Knife and others) as well as a ba